# Въведение в вероятността и статистиката
В тази тетрадка ще се поразиграем с някои от концепциите, които вече обсъждахме. Много концепции от вероятността и статистиката са добре представени в основните библиотеки за обработка на данни в Python, като `numpy` и `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Случайни променливи и разпределения
Нека започнем с изтегляне на проба от 30 стойности от еднородно разпределение от 0 до 9. Ще изчислим и средната стойност и дисперсията.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

За да оценим визуално колко различни стойности има в извадката, можем да начертаем **хистограма**:


In [ ]:
plt.hist(sample)
plt.show()

## Анализ на реални данни

Средната стойност и дисперсията са много важни при анализа на реални данни. Нека заредим данните за бейзболистите от [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Използваме пакет, наречен [**Pandas**](https://pandas.pydata.org/) тук за анализ на данни. Ще говорим повече за Pandas и работа с данни в Python по-нататък в този курс.

Нека изчислим средните стойности за възраст, ръст и тегло:


In [ ]:
df[['Age','Height','Weight']].mean()

Сега нека се съсредоточим върху височината и изчислим стандартното отклонение и дисперсията:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

В допълнение към средната стойност, има смисъл да се разгледа медианата и квартили. Те могат да бъдат визуализирани с помощта на **box plot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Можем също да направим кутии диаграми на подмножества от нашия набор от данни, например групирани по роля на играча.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Бележка**: Тази диаграма подсказва, че средно височините на първите бейзмени са по-големи от височините на вторите бейзмени. По-късно ще научим как можем да тестваме тази хипотеза по-формално и как да демонстрираме, че нашите данни са статистически значими, за да го покажем.  

Възрастта, височината и теглото са всички непрекъснати случайни променливи. Как мислите какво е тяхното разпределение? Добър начин да разберете е да начертаете хистограма на стойностите: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Нормално разпределение

Нека създадем изкуствена извадка от тежести, която следва нормално разпределение със същата средна стойност и дисперсия като нашите реални данни:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Тъй като повечето стойности в реалния живот са нормално разпределени, не трябва да използваме равномерно генератор на случайни числа за генериране на примерни данни. Ето какво се случва, ако се опитаме да генерираме тегла с равномерно разпределение (генерирано от `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Интервали на доверие

Нека сега изчислим интервали на доверие за теглата и височините на бейзболистите. Ще използваме кода [от тази дискусия в stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Тестване на хипотези

Нека изследваме различни роли в нашия набор от данни за бейзболни играчи:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Нека тестваме хипотезата, че първите бейзболисти са по-високи от вторите бейзболисти. Най-простият начин да направим това е да тестваме интервалите на доверие:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Можем да видим, че интервалите не се припокриват.

Статистически по-коректен начин да докажем хипотезата е да използваме **тест на Стюдент (Student t-test)**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Двете стойности, върнати от функцията `ttest_ind`, са:
* p-стойността може да се разглежда като вероятността две разпределения да имат еднаква средна стойност. В нашия случай тя е много ниска, което означава, че има силни доказателства, подкрепящи, че първите застъпници са по-високи.
* t-стойността е междинната стойност на нормализираната разлика на средните, която се използва в t-теста и се сравнява с прагова стойност за дадена стойност на доверие.


## Симулиране на нормално разпределение с централната гранична теорема

Псевдо-случайният генератор в Python е проектиран да ни даде равно разпределение. Ако искаме да създадем генератор за нормално разпределение, можем да използваме централната гранична теорема. За да получим стойност с нормално разпределение, просто ще изчислим средноаритметичното на проба, генерирана с равно разпределение.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Корелация и Evil Baseball Corp

Корелацията ни позволява да намерим връзки между поредици от данни. В нашия опростен пример, нека си представим, че съществува зла бейзболна корпорация, която плаща на играчите си според техния ръст - колкото по-висок е играчът, толкова повече пари получава. Да приемем, че има базова заплата от 1000 долара и допълнителен бонус от 0 до 100 долара, в зависимост от ръста. Ще вземем реалните играчи от MLB и ще изчислим техните въображаеми заплати:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Нека сега изчислим коварията и корелацията на тези последователности. `np.cov` ще ни даде така наречената **ковариационна матрица**, която е разширение на коварията за множество променливи. Елементът $M_{ij}$ на коварияната матрица $M$ е корелацията между входните променливи $X_i$ и $X_j$, а диагоналните стойности $M_{ii}$ са дисперсията на $X_{i}$. По същия начин `np.corrcoef` ще ни даде **корелационна матрица**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Корелация равна на 1 означава, че има силна **линейна връзка** между две променливи. Можем визуално да видим линейната връзка, като начертаем една стойност срещу другата:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Нека видим какво се случва, ако връзката не е линейна. Да приемем, че нашата корпорация е решила да скрие очевидната линейна зависимост между ръста и заплатите, и е въвела малко нелинейност във формулата, като например `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

В този случай корелацията е малко по-малка, но все още е доста висока. Сега, за да направим връзката още по-малко очевидна, може да искаме да добавим допълнителна случайност, като добавим някаква случайна променлива към заплатата. Нека видим какво ще се случи:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Можете ли да предположите защо точките се подреждат в вертикални линии по този начин?

Наблюдавахме корелацията между изкуствено конструирана концепция като заплата и наблюдавана променлива *ръст*. Нека видим и дали двете наблюдавани променливи, като ръст и тегло, също корелират:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

За съжаление, не получихме никакви резултати - само някои странни стойности `nan`. Това се дължи на факта, че някои от стойностите в нашата серия са неопределени, представени като `nan`, което води до това резултатът от операцията също да е неопределен. Като погледнем матрицата, можем да видим, че колоната `Weight` е проблематична, защото е изчислена самокорелация между стойностите на `Height`.

> Този пример показва значението на **подготовката на данните** и **почистването** им. Без подходящи данни не можем да изчислим нищо.

Нека използваме метода `fillna`, за да запълним липсващите стойности и да изчислим корелацията: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Наистина съществува корелация, но не така силна както в нашия изкуствен пример. Всъщност, ако погледнем разсейващия график на една стойност спрямо другата, връзката би била много по-малко очевидна:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Заключение

В този бележник научихме как да изпълняваме основни операции с данни за изчисляване на статистически функции. Сега знаем как да използваме солиден арсенал от математика и статистика, за да докажем някои хипотези и как да изчисляваме интервали на доверие за произволни променливи, като използваме дадена проба от данни. 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:  
Този документ е преведен с помощта на AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за възникнали недоразумения или неправилни тълкувания, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
